**Task 07: Querying RDF(s)**

In [16]:
!pip install rdflib
import urllib.request
url = 'https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/refs/heads/master/Assignment4/course_materials/python/validation.py'
urllib.request.urlretrieve(url, 'validation.py')
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/master/Assignment4/course_materials"


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from validation import Report

First let's read the RDF file

In [18]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
# Do not change the name of the variables
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.parse(github_storage+"/rdf/data06.ttl", format="TTL")
report = Report()

**TASK 7.1a: For all classes, list each classURI. If the class belogs to another class, then list its superclass.**
**Do the exercise in RDFLib returning a list of Tuples: (class, superclass) called "result". If a class does not have a super class, then return None as the superclass**

In [19]:
# TO DO
# Visualize the results
result = [] #list of tuples
for s in g.subjects(RDF.type, RDFS.Class):
    superclass = g.value(s, RDFS.subClassOf)
    result.append((s, superclass))
for r in result:
  print(r)

(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Animal'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Student'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#FullProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#InterimAssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'))


In [20]:
## Validation: Do not remove
report.validate_07_1a(result)

TASK 7.1a OK


**TASK 7.1b: Repeat the same exercise in SPARQL, returning the variables ?c (class) and ?sc (superclass)**

In [21]:
query =  """SELECT DISTINCT ?c ?sc WHERE{
?c a rdfs:Class.
OPTIONAL{?c rdfs:subClassOf ?sc}.
}"""

for r in g.query(query):
  print(r.c, r.sc)

http://oeg.fi.upm.es/def/people#Person None
http://oeg.fi.upm.es/def/people#Animal None
http://oeg.fi.upm.es/def/people#Professor http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#Student http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#FullProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#AssociateProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#InterimAssociateProfessor http://oeg.fi.upm.es/def/people#AssociateProfessor


In [22]:
## Validation: Do not remove
report.validate_07_1b(query,g)

TASK 7.1b OK


**TASK 7.2a: List all individuals of "Person" with RDFLib (remember the subClasses). Return the individual URIs in a list called "individuals"**


In [23]:
ns = Namespace("http://oeg.fi.upm.es/def/people#")

# variable to return
individuals = []


revisar = [ns.Person]
revisado = set()

while revisar:
    actual = revisar.pop()
    revisado.add(actual)
    for sc in g.subjects(RDFS.subClassOf, actual):
        if sc not in revisado:
            revisar.append(sc)


for cl in revisado:
    for individuo in g.subjects(RDF.type, cl):
        individuals.append(individuo)

In [24]:
# validation. Do not remove
report.validate_07_02a(individuals)

TASK 7.2a OK


**TASK 7.2b: Repeat the same exercise in SPARQL, returning the individual URIs in a variable ?ind**

In [25]:
query =  """SELECT DISTINCT ?ind WHERE{
?ind a ?class .
?class rdfs:subClassOf* <http://oeg.fi.upm.es/def/people#Person>.
}"""

for r in g.query(query):
  print(r.ind)

http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar
http://oeg.fi.upm.es/def/people#Raul


In [26]:
## Validation: Do not remove
report.validate_07_02b(g, query)

TASK 7.2b OK


**TASK 7.3:  List the name and type of those who know Rocky (in SPARQL only). Use name and type as variables in the query**

In [27]:
query =  """SELECT ?name ?type WHERE{
    ?name <http://oeg.fi.upm.es/def/people#knows> <http://oeg.fi.upm.es/def/people#Rocky>.
    ?name a ?type
  }"""
# Visualize the results
for r in g.query(query):
  print(r.name, r.type)


http://oeg.fi.upm.es/def/people#Asun http://oeg.fi.upm.es/def/people#FullProfessor
http://oeg.fi.upm.es/def/people#Raul http://oeg.fi.upm.es/def/people#InterimAssociateProfessor
http://oeg.fi.upm.es/def/people#Fantasma http://oeg.fi.upm.es/def/people#Animal


In [28]:
## Validation: Do not remove
report.validate_07_03(g, query)

TASK 7.3 OK


**Task 7.4: List the name of those entities who have a colleague with a dog, or that have a collegue who has a colleague who has a dog (in SPARQL). Return the results in a variable called name**

In [29]:
query =  """SELECT ?name WHERE {
  ?p1 <http://oeg.fi.upm.es/def/people#ownsPet> ?pet.
  {?name <http://oeg.fi.upm.es/def/people#hasColleague> ?p1}
  UNION
  {?p2 <http://oeg.fi.upm.es/def/people#hasColleague> ?p1. ?name <http://oeg.fi.upm.es/def/people#hasColleague> ?p2.}
}
"""
# TO DO
# Visualize the results

In [30]:
## Validation: Do not remove
report.validate_07_04(g,query)
report.save_report("_Task_07")

TASK 7.4 OK
